In [ ]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
import json
import os
from datetime import datetime

# ==========================
# HÀM PHỤ TÍNH TOÁN
# ==========================

def compute_vs(p1, p2, v_f, v_AUV):
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    Lx, Ly, Lz = x2 - x1, y2 - y1, z2 - z1
    L_mag = math.sqrt(Lx**2 + Ly**2 + Lz**2)
    if L_mag == 0:
        return v_AUV
    cos_beta = Lz / L_mag
    cos_beta = np.clip(cos_beta, -1, 1)
    beta = math.acos(cos_beta)
    inner = np.clip((v_f * cos_beta) / v_AUV, -1, 1)
    angle = beta + math.acos(inner)
    v_s = abs(math.cos(angle) * v_AUV / cos_beta)
    return v_s


def travel_time(path, coords, v_f, v_AUV):
    total_time = 0.0
    for i in range(len(path) - 1):
        p1, p2 = coords[path[i]], coords[path[i + 1]]
        d = np.linalg.norm(np.array(p2) - np.array(p1))
        v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
        total_time += d / max(v_s, 1e-9)
    # quay lại điểm đầu
    p1, p2 = coords[path[-1]], coords[path[0]]
    d = np.linalg.norm(np.array(p2) - np.array(p1))
    v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
    total_time += d / max(v_s, 1e-9)
    return total_time


# ==========================
# LỚP CHÍNH GA
# ==========================

class ClusterTSP_GA:
    def __init__(self, clusters, ga_params=None):
        self.clusters = clusters
        self.cluster_centers = [(0.0, 0.0, 0.0)] + [clusters[k]["center"] for k in sorted(clusters.keys())]
        self.n = len(self.cluster_centers)

        defaults = {
            'pop_size': 50,
            'generations': 200,
            'crossover_rate': 0.8,
            'mutation_rate': 0.2,
            'elitism_k': 3,
            'tournament_size': 3,
            'crossover_type': 'OX',
            'mutation_type': 'inversion',
            'local_search': True,
            'v_f': 0.3,
            'v_AUV': 1.0,
            'verbose': False
        }
        if ga_params:
            defaults.update(ga_params)
        self.params = defaults
        self.best_fitness_history = []

    def create_individual(self):
        seq = list(range(1, self.n))
        random.shuffle(seq)
        return [0] + seq

    def create_population(self):
        return [self.create_individual() for _ in range(self.params['pop_size'])]

    def fitness(self, ind):
        total_time = travel_time(ind, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
        return 1.0 / (total_time + 1e-9)

    def tournament_selection(self, population):
        return max(random.sample(population, self.params['tournament_size']), key=self.fitness)

    def order_crossover(self, p1, p2):
        sub1, sub2 = p1[1:], p2[1:]
        a, b = sorted(random.sample(range(len(sub1)), 2))
        c1, c2 = [-1]*len(sub1), [-1]*len(sub1)
        c1[a:b], c2[a:b] = sub1[a:b], sub2[a:b]
        ptr = b
        for x in sub2[b:]+sub2[:b]:
            if x not in c1:
                c1[ptr % len(sub1)] = x
                ptr += 1
        ptr = b
        for x in sub1[b:]+sub1[:b]:
            if x not in c2:
                c2[ptr % len(sub2)] = x
                ptr += 1
        return [0]+c1, [0]+c2

    def inversion_mutation(self, ind):
        i, j = sorted(random.sample(range(1, len(ind)), 2))
        ind[i:j+1] = list(reversed(ind[i:j+1]))
        return ind

    def evolve(self):
        pop = self.create_population()
        best = max(pop, key=self.fitness)
        best_time = travel_time(best, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
        for _ in range(self.params['generations']):
            fitnesses = [self.fitness(ind) for ind in pop]
            best_gen = pop[np.argmax(fitnesses)]
            gen_best_time = travel_time(best_gen, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
            if gen_best_time < best_time:
                best, best_time = best_gen.copy(), gen_best_time
            elite_idx = np.argsort(fitnesses)[-self.params['elitism_k']:]
            new_pop = [pop[i].copy() for i in elite_idx]
            while len(new_pop) < self.params['pop_size']:
                p1, p2 = self.tournament_selection(pop), self.tournament_selection(pop)
                c1, c2 = self.order_crossover(p1, p2) if random.random() < self.params['crossover_rate'] else (p1.copy(), p2.copy())
                if random.random() < self.params['mutation_rate']:
                    c1 = self.inversion_mutation(c1)
                if random.random() < self.params['mutation_rate']:
                    c2 = self.inversion_mutation(c2)
                new_pop += [c1, c2]
            pop = new_pop[:self.params['pop_size']]
        return best, best_time


def compute_energy(best_time):
    G, L, n = 100, 1024, 4
    P_t, P_r, P_idle, DR, DR_i = 1.6e-3, 0.8e-3, 0.1e-3, 4000, 1e6
    E_tx_MN = G * P_t * L / DR
    E_idle_MN = (best_time - G * L / DR) * P_idle
    E_total_MN = E_tx_MN + E_idle_MN
    E_rx_TN = G * P_r * L * n / DR
    E_tx_TN = G * P_t * L * n / DR_i
    E_idle_TN = (best_time - (G * L * n / DR) - (G * L * n / DR_i)) * P_idle
    E_total_TN = E_rx_TN + E_tx_TN + E_idle_TN
    return {
        "Member": {"E_tx": E_tx_MN, "E_idle": E_idle_MN, "E_total": E_total_MN},
        "Target": {"E_rx": E_rx_TN, "E_tx": E_tx_TN, "E_idle": E_idle_TN, "E_total": E_total_TN}
    }


# ==========================
# CHẠY TOÀN BỘ CÁC FILE JSON TRONG THƯ MỤC
# ==========================

def main():
    input_dir = "D:/Year 4/tiến hóa/project/UWSN_greedy/output_data_kmeans"
    output_dir = "D:/Year 4/tiến hóa/project/UWSN_greedy/output_path/output_ga/"

    os.makedirs(output_dir, exist_ok=True)
    files = [f for f in os.listdir(input_dir) if f.endswith(".json")]

    ga_params = {
        'pop_size': 40,
        'generations': 200,
        'crossover_rate': 0.8,
        'mutation_rate': 0.2,
        'elitism_k': 3,
        'local_search': True,
        'v_f': 1.2,
        'v_AUV': 3.0,
        'verbose': False
    }

    for filename in files:
        input_path = os.path.join(input_dir, filename)
        print(f"\n=== Đang xử lý file: {filename} ===")

        with open(input_path, 'r') as f:
            clusters = json.load(f)

        ga = ClusterTSP_GA(clusters, ga_params)
        best, best_time = ga.evolve()
        energy = compute_energy(best_time)

        result = {
            "input_file": filename,
            "best_path": best,
            "best_time": best_time,
            "energy": energy,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        output_path = os.path.join(output_dir, f"result_{os.path.splitext(filename)[0]}.json")
        with open(output_path, 'w') as f:
            json.dump(result, f, indent=4)

        print(f"Đã lưu kết quả: {output_path}")
        print(f"   -> Best time: {best_time:.4f}s\n")


if __name__ == '__main__':
    main()


In [ ]:
#new
import numpy as np
import random
import math
import matplotlib.pyplot as plt
import json
import os
from datetime import datetime
from sklearn.cluster import KMeans

# ==========================
# HÀM TÍNH TOÁN TỐC ĐỘ & THỜI GIAN
# ==========================

def compute_vs(p1, p2, v_f, v_AUV):
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    Lx, Ly, Lz = x2 - x1, y2 - y1, z2 - z1
    L_mag = math.sqrt(Lx**2 + Ly**2 + Lz**2)
    if L_mag == 0:
        return v_AUV
    cos_beta = Lz / L_mag
    cos_beta = np.clip(cos_beta, -1, 1)
    beta = math.acos(cos_beta)
    inner = np.clip((v_f * cos_beta) / v_AUV, -1, 1)
    angle = beta + math.acos(inner)
    v_s = abs(math.cos(angle) * v_AUV / cos_beta)
    return v_s


def travel_time(path, coords, v_f, v_AUV):
    total_time = 0.0
    for i in range(len(path) - 1):
        p1, p2 = coords[path[i]], coords[path[i + 1]]
        d = np.linalg.norm(np.array(p2) - np.array(p1))
        v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
        total_time += d / max(v_s, 1e-9)
    # quay lại điểm đầu
    p1, p2 = coords[path[-1]], coords[path[0]]
    d = np.linalg.norm(np.array(p2) - np.array(p1))
    v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
    total_time += d / max(v_s, 1e-9)
    return total_time


# ==========================
# LỚP GA (đã chỉnh để map index -> cluster_head id)
# ==========================

class ClusterTSP_GA:
    def __init__(self, clusters, ga_params=None):
        # clusters: dict {cid: {"nodes": [...], "center": [x,y,z], "cluster_head": nid}}
        self.clusters = clusters
        # build ordered list of centers and mapping index->cluster_head id
        sorted_keys = sorted(clusters.keys(), key=lambda x: int(x))
        self.index_to_ch = [None]  # index 0 is AUV base (O)
        self.cluster_centers = [(0.0, 0.0, 0.0)]
        for k in sorted_keys:
            c = clusters[k]['center']
            self.cluster_centers.append(tuple(c))
            self.index_to_ch.append(clusters[k].get('cluster_head', None))

        self.n = len(self.cluster_centers)

        defaults = {
            'pop_size': 50,
            'generations': 200,
            'crossover_rate': 0.8,
            'mutation_rate': 0.2,
            'elitism_k': 3,
            'tournament_size': 3,
            'crossover_type': 'OX',
            'mutation_type': 'inversion',
            'local_search': True,
            'v_f': 1.2,
            'v_AUV': 3.0,
            'verbose': False
        }
        if ga_params:
            defaults.update(ga_params)
        self.params = defaults
        self.best_fitness_history = []

    def create_individual(self):
        seq = list(range(1, self.n))
        random.shuffle(seq)
        return [0] + seq

    def create_population(self):
        return [self.create_individual() for _ in range(self.params['pop_size'])]

    def fitness(self, ind):
        total_time = travel_time(ind, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
        return 1.0 / (total_time + 1e-9)

    def tournament_selection(self, population):
        return max(random.sample(population, self.params['tournament_size']), key=self.fitness)

    def order_crossover(self, p1, p2):
        sub1, sub2 = p1[1:], p2[1:]
        a, b = sorted(random.sample(range(len(sub1)), 2))
        c1, c2 = [-1]*len(sub1), [-1]*len(sub1)
        c1[a:b], c2[a:b] = sub1[a:b], sub2[a:b]
        ptr = b
        for x in sub2[b:]+sub2[:b]:
            if x not in c1:
                c1[ptr % len(sub1)] = x
                ptr += 1
        ptr = b
        for x in sub1[b:]+sub1[:b]:
            if x not in c2:
                c2[ptr % len(sub2)] = x
                ptr += 1
        return [0]+c1, [0]+c2

    def inversion_mutation(self, ind):
        i, j = sorted(random.sample(range(1, len(ind)), 2))
        ind[i:j+1] = list(reversed(ind[i:j+1]))
        return ind

    def evolve(self):
        pop = self.create_population()
        best = max(pop, key=self.fitness)
        best_time = travel_time(best, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
        for _ in range(self.params['generations']):
            fitnesses = [self.fitness(ind) for ind in pop]
            best_gen = pop[np.argmax(fitnesses)]
            gen_best_time = travel_time(best_gen, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
            if gen_best_time < best_time:
                best, best_time = best_gen.copy(), gen_best_time
            elite_idx = np.argsort(fitnesses)[-self.params['elitism_k']:]
            new_pop = [pop[i].copy() for i in elite_idx]
            while len(new_pop) < self.params['pop_size']:
                p1, p2 = self.tournament_selection(pop), self.tournament_selection(pop)
                c1, c2 = self.order_crossover(p1, p2) if random.random() < self.params['crossover_rate'] else (p1.copy(), p2.copy())
                if random.random() < self.params['mutation_rate']:
                    c1 = self.inversion_mutation(c1)
                if random.random() < self.params['mutation_rate']:
                    c2 = self.inversion_mutation(c2)
                new_pop += [c1, c2]
            pop = new_pop[:self.params['pop_size']]
        # convert best (indices) to cluster head node ids sequence (map 0->'O')
        mapped_path = ['O' if idx == 0 else self.index_to_ch[idx] for idx in best]
        return best, mapped_path, best_time


# ==========================
# HÀM NĂNG LƯỢNG (giữ nguyên công thức bạn cung cấp)
# ==========================

def compute_energy(best_time):
    G, L, n = 100, 1024, 4
    P_t, P_r, P_idle, DR, DR_i = 1.6e-3, 0.8e-3, 0.1e-3, 4000, 1e6
    E_tx_MN = G * P_t * L / DR
    E_idle_MN = (best_time - G * L / DR) * P_idle
    E_total_MN = E_tx_MN + E_idle_MN
    E_rx_TN = G * P_r * L * n / DR
    E_tx_TN = G * P_t * L * n / DR_i
    E_idle_TN = (best_time - (G * L * n / DR) - (G * L * n / DR_i)) * P_idle
    E_total_TN = E_rx_TN + E_tx_TN + E_idle_TN
    return {
        "Member": {"E_tx": E_tx_MN, "E_idle": E_idle_MN, "E_total": E_total_MN},
        "Target": {"E_rx": E_rx_TN, "E_tx": E_tx_TN, "E_idle": E_idle_TN, "E_total": E_total_TN}
    }


# ==========================
# HÀM PHÂN CỤM VÀ CHỌN CLUSTER HEAD (sử dụng code bạn cung cấp)
# ==========================
def split_isolated_nodes(clusters, r_sen=60, node_data=None):
    """
    Tách node dựa vào khoảng cách tới Cluster Head, không dùng tâm cụm.
    Node nào xa hơn r_sen → tách thành cụm riêng.
    """

    new_clusters = []

    for c in clusters:
        nodes = c["nodes"]
        ids = c["node_ids"]

        # Lấy cluster head thật sự (không phải center)
        head_id = choose_cluster_head(c, node_data)
        head_index = ids.index(head_id)
        head_pos = nodes[head_index]

        # Tính khoảng cách node tới cluster head
        dists = np.linalg.norm(nodes - head_pos, axis=1)

        # Node trong cụm hợp lệ (gần cluster head)
        in_cluster_idx = np.where(dists <= r_sen)[0]
        # Node bị tách vì xa head
        isolated_idx = np.where(dists > r_sen)[0]

        # Cụm giữ lại (có head và các node gần head)
        if len(in_cluster_idx) > 0:
            new_clusters.append({
                "node_ids": [ids[i] for i in in_cluster_idx],
                "nodes": nodes[in_cluster_idx],
                "center": np.mean(nodes[in_cluster_idx], axis=0),
                "node_data": {nid: c["node_data"][nid] for nid in [ids[i] for i in in_cluster_idx]},
                "cluster_head": head_id   # Lưu head luôn
            })

        # Node bị tách → mỗi node thành cụm riêng
        for i in isolated_idx:
            nid = ids[i]
            node = nodes[i]
            new_clusters.append({
                "node_ids": [nid],
                "nodes": np.array([node]),
                "center": node,
                "node_data": {nid: c["node_data"][nid]},
                "cluster_head": nid  # node lẻ tự làm head
            })

    return new_clusters
def cluster_split(nodes, node_ids, node_data=None, r_sen=60, R=150, max_depth=50, depth=0):
    """
    Hàm phân cụm đệ quy (Algorithm 1) + giữ danh tính node lẻ (isolated).
    """

    center = np.mean(nodes, axis=0)
    dists = np.linalg.norm(nodes - center, axis=1)

    # Điều kiện dừng
    if (len(nodes) <= R and np.all(dists <= r_sen)) or depth >= max_depth:
        return [{
            "node_ids": node_ids,
            "nodes": nodes,
            "center": center,
            "node_data": node_data if node_data else {}
        }]

    # weights for distance vs energy
    w_dist = 0.5
    w_energy = 0.5

    # coords -> chuẩn hóa theo phạm vi hiện tại trong cụm
    coords = nodes.astype(float)
    coord_scale = np.ptp(coords, axis=0).max()  # peak-to-peak của x,y,z
    if coord_scale <= 0:
        coords_norm = np.zeros_like(coords)
    else:
        coords_norm = coords / (coord_scale + 1e-9)

    # energy feature: lấy residual_energy từ node_data nếu có, else mặc định
    if node_data:
        energies = np.array([node_data.get(nid, {}).get('residual_energy', 100.0) for nid in node_ids], dtype=float)
    else:
        energies = np.full(len(node_ids), 100.0, dtype=float)

    # chuẩn hóa năng lượng và đảo chiều nếu muốn năng lượng cao => ưu tiên (giá trị nhỏ hơn)
    emin, emax = energies.min(), energies.max()
    if emax - emin < 1e-9:
        e_norm = np.full_like(energies, 0.5)
    else:
        e_norm = 1.0 - (energies - emin) / (emax - emin)  # in [0,1]; năng lượng lớn => giá trị nhỏ

    # ghép feature với trọng số (dùng sqrt để trọng số tương ứng dưới Euclidean)
    feat = np.hstack([
        coords_norm * np.sqrt(w_dist),
        e_norm.reshape(-1, 1) * np.sqrt(w_energy)
    ])

    # nếu không đủ mẫu để chia tiếp (1 node) thì gán nhãn 0
    if len(node_ids) < 2:
        labels = np.zeros(len(node_ids), dtype=int)
    else:
        kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
        labels = kmeans.fit_predict(feat)

    clusters = []
    for i in range(2):
        sub_nodes = nodes[labels == i]
        sub_ids = [node_ids[j] for j in range(len(node_ids)) if labels[j] == i]

        sub_data = {nid: node_data[nid] for nid in sub_ids} if node_data else None

        clusters += cluster_split(sub_nodes, sub_ids, sub_data, r_sen, R, max_depth, depth + 1)

    # TÁCH NODE LẺ SAU KHI ĐỆ QUY
    clusters = split_isolated_nodes(clusters, r_sen=r_sen, node_data=node_data)

    return clusters


def choose_cluster_head(cluster, node_data_dict):
    nodes = cluster["nodes"]
    center = cluster["center"]
    node_ids = cluster["node_ids"]

    dists_to_center = np.linalg.norm(nodes - center, axis=1)
    max_Q = -1
    best_cluster_head = node_ids[0]

    for i, node_id in enumerate(node_ids):
        if node_id in node_data_dict:
            node_info = node_data_dict[node_id]
            E_current = node_info.get('residual_energy', 100.0)
            E0 = node_info.get('initial_energy', 100.0)
            if E_current <= 0:
                E_current = 0.1
            energy_ratio = E0 / E_current
            d_tocenter = dists_to_center[i]
            Q = d_tocenter ** energy_ratio
            if Q > max_Q:
                max_Q = Q
                best_cluster_head = node_id
        else:
            # fallback: choose nearest to center
            if i == 0 or dists_to_center[i] < dists_to_center[node_ids.index(best_cluster_head)]:
                best_cluster_head = node_id
    return best_cluster_head


# ==========================
# HÀM QUẢN LÝ NĂNG LƯỢNG VÀ NODE CHẾT
# ==========================

def update_energy(all_nodes, clusters, energy_report):
    # energy_report has 'Member' and 'Target' E_total
    for cid, cinfo in clusters.items():
        ch = cinfo.get('cluster_head')
        nodes = cinfo.get('nodes', [])
        for nid in nodes:
            if nid not in all_nodes:
                continue
            if nid == ch:
                all_nodes[nid]['residual_energy'] -= energy_report['Target']['E_total']
            else:
                all_nodes[nid]['residual_energy'] -= energy_report['Member']['E_total']
            # clamp
            if all_nodes[nid]['residual_energy'] < 0:
                all_nodes[nid]['residual_energy'] = 0.0


def remove_dead_nodes(all_nodes, clusters):
    dead = [nid for nid, info in list(all_nodes.items()) if info['residual_energy'] <= 0]
    for nid in dead:
        del all_nodes[nid]
    # remove from node_positions will be handled by caller
    # clean clusters: remove clusters that have no nodes left
    new_clusters = {}
    for cid, cinfo in clusters.items():
        alive_nodes = [nid for nid in cinfo.get('nodes', []) if nid in all_nodes]
        if alive_nodes:
            new_c = dict(cinfo)
            new_c['nodes'] = alive_nodes
            # if cluster head died, will select new CH during recluster
            new_clusters[cid] = new_c
    return new_clusters, dead


# ==========================
# HÀM PHÂN CỤM LẠI TỪ node_positions
# ==========================

def recluster(all_nodes, node_positions, r_sen=50, R=20):
    ids = sorted(list(all_nodes.keys()))
    if len(ids) == 0:
        return {}
    coords = np.array([node_positions[nid] for nid in ids])
    raw_clusters = cluster_split(coords, ids, all_nodes, r_sen=r_sen, R=R)
    clusters = {}
    for i, c in enumerate(raw_clusters):
        center = c['center'].tolist()
        node_ids = c['node_ids']
        ch = choose_cluster_head(c, all_nodes)
        clusters[i] = {'nodes': node_ids, 'center': center, 'cluster_head': ch}
    return clusters


# ==========================
# MAIN: vòng lặp nhiều chu kỳ
# ==========================

def main():
    input_dir = "D:/Year 4/tiến hóa/project/data/output_data_kmeans"
    output_dir = "D:/Year 4/tiến hóa/project/data/output_path/output_ga_multicycle"
    os.makedirs(output_dir, exist_ok=True)

    files = [f for f in os.listdir(input_dir) if f.endswith('.json')]

    ga_params = {
        'pop_size': 40,
        'generations': 200,
        'crossover_rate': 0.8,
        'mutation_rate': 0.2,
        'elitism_k': 3,
        'local_search': True,
        'v_f': 1.2,
        'v_AUV': 3.0,
        'verbose': False
    }

    INITIAL_ENERGY = 100.0

    for filename in files:
        input_path = os.path.join(input_dir, filename)
        print(f"\n=== Đang xử lý file: {filename} ===")
        with open(input_path, 'r') as f:
            clusters_in = json.load(f)

        # Build initial node list and try to find node positions
        # If there's a separate nodes_positions.json in same folder, use it.
        # Else, approximate member node positions by using cluster centers + small noise.
        node_positions = {}
        all_nodes = {}

        # collect all node ids from clusters_in
        all_node_ids = set()
        for k, v in clusters_in.items():
            for nid in v.get('nodes', []):
                all_node_ids.add(nid)
            # cluster_head too (may duplicate)
            ch = v.get('cluster_head')
            if ch is not None:
                all_node_ids.add(ch)

        # attempt to load node positions file (optional)
        nodes_pos_file = "D:/Year 4/tiến hóa/project/data/input_data_evenly_distributed/nodes_20.json"
        if os.path.exists(nodes_pos_file):
            try:
                with open(nodes_pos_file, 'r', encoding='utf-8') as f:
                    nodes_data = json.load(f)
            
                # Parse từ list of dict sang dict với key là node id
                node_positions = {}
                for node in nodes_data:
                    node_id = node['id']
                    node_positions[node_id] = (node['x'], node['y'], node['z'])
                
                print(f"Đã nạp node_positions từ {os.path.basename(nodes_pos_file)}")
                print(f"Số lượng node positions: {len(node_positions)}")
            except Exception as e:
                print(f"Không thể đọc {os.path.basename(nodes_pos_file)}, sẽ tạo vị trí gần center. Error:", e)

        # if no node_positions available, approximate
        if not node_positions:
            for k, v in clusters_in.items():
                center = tuple(v.get('center', (0.0, 0.0, 0.0)))
                for nid in v.get('nodes', []):
                    # small random offset
                    offset = np.random.normal(scale=5.0, size=3)
                    node_positions[nid] = tuple(np.array(center) + offset)
                # ensure cluster head has a position (if listed separately)
                ch = v.get('cluster_head')
                if ch is not None and ch not in node_positions:
                    node_positions[ch] = center
            print("Tạo giả lập node_positions bằng center + noise (vì không tìm thấy file vị trí)")

        # initialize energy
        for nid in list(all_node_ids):
            all_nodes[nid] = {'initial_energy': INITIAL_ENERGY, 'residual_energy': INITIAL_ENERGY}

        total_nodes = len(all_nodes)
        print(f"Tổng số node ban đầu: {total_nodes}")

        # Start with clusters_in (possibly reformat keys to ints)
        clusters = {}
        for k, v in clusters_in.items():
            clusters[int(k)] = {'nodes': v.get('nodes', []), 'center': v.get('center', []), 'cluster_head': v.get('cluster_head')}

        cycle = 0
        outputs = []

        while True:
            cycle += 1
            print(f"\n--- Cycle {cycle} ---")

            alive_ratio = len(all_nodes) / total_nodes if total_nodes > 0 else 0
            print(f"Tỉ lệ node sống: {alive_ratio*100:.2f}% ({len(all_nodes)}/{total_nodes})")
            if alive_ratio < 0.9:
                print("Dừng mô phỏng: tỉ lệ node sống < 90%")
                break

            if len(clusters) == 0:
                print("Không còn cụm nào (không còn node). Dừng.")
                break

            # run GA on current clusters
            ga = ClusterTSP_GA(clusters, ga_params)
            best_indices, best_mapped_path, best_time = ga.evolve()

            energy_report = compute_energy(best_time)

            # update energies
            update_energy(all_nodes, clusters, energy_report)

            # remove dead nodes
            clusters, dead_nodes = remove_dead_nodes(all_nodes, clusters)
            # also remove from node_positions
            for d in dead_nodes:
                if d in node_positions:
                    del node_positions[d]

            # log output for this cycle
            outputs.append({
                'cycle': cycle,
                'num_clusters': len(clusters),
                'best_path_indices': best_indices,
                'best_path_node_ids': best_mapped_path,
                'best_time': best_time,
                'dead_nodes': dead_nodes,
                'alive_nodes': len(all_nodes)
            })

            print(f"Số cụm hiện tại: {len(clusters)}")
            print(f"Đường đi (index trong GA): {best_indices}")
            print(f"Đường đi (node ids, 'O' = base): {best_mapped_path}")
            if dead_nodes:
                print("Nodes bị loại (chết) trong chu kỳ này:", dead_nodes)
            else:
                print("Không có node chết ở chu kỳ này.")

            # if still nodes left, recluster and choose CH for next cycle
            if len(all_nodes) > 0:
                clusters = recluster(all_nodes, node_positions)
                # if recluster produced clusters, ensure centers are lists
                for k, v in clusters.items():
                    clusters[k]['center'] = [float(x) for x in v['center']]

        # save outputs to file
        out_filename = os.path.join(output_dir, f"multicycle_result_{os.path.splitext(filename)[0]}.json")
        meta = {
            'input_file': filename,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'initial_total_nodes': total_nodes,
            'cycles': cycle - 1,
            'outputs': outputs
        }
        with open(out_filename, 'w') as f:
            json.dump(meta, f, indent=4)

        print(f"Kết quả đã lưu: {out_filename}")


if __name__ == '__main__':
    main()


In [ ]:
#claude
import numpy as np
import random
import math
import matplotlib.pyplot as plt
import json
import os
from datetime import datetime
from sklearn.cluster import KMeans

# ==========================
# HÀM TÍNH TOÁN TỐC ĐỘ & THỜI GIAN
# ==========================

def compute_vs(p1, p2, v_f, v_AUV):
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    Lx, Ly, Lz = x2 - x1, y2 - y1, z2 - z1
    L_mag = math.sqrt(Lx**2 + Ly**2 + Lz**2)
    if L_mag == 0:
        return v_AUV
    cos_beta = Lz / L_mag
    cos_beta = np.clip(cos_beta, -1, 1)
    beta = math.acos(cos_beta)
    inner = np.clip((v_f * cos_beta) / v_AUV, -1, 1)
    angle = beta + math.acos(inner)
    v_s = abs(math.cos(angle) * v_AUV / cos_beta)
    return v_s


def travel_time(path, coords, v_f, v_AUV):
    total_time = 0.0
    for i in range(len(path) - 1):
        p1, p2 = coords[path[i]], coords[path[i + 1]]
        d = np.linalg.norm(np.array(p2) - np.array(p1))
        v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
        total_time += d / max(v_s, 1e-9)
    # quay lại điểm đầu
    p1, p2 = coords[path[-1]], coords[path[0]]
    d = np.linalg.norm(np.array(p2) - np.array(p1))
    v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
    total_time += d / max(v_s, 1e-9)
    return total_time


# ==========================
# LỚP GA (đã chỉnh để map index -> cluster_head id)
# ==========================

class ClusterTSP_GA:
    def __init__(self, clusters, ga_params=None):
        # clusters: dict {cid: {"nodes": [...], "center": [x,y,z], "cluster_head": nid}}
        self.clusters = clusters
        # build ordered list of centers and mapping index->cluster_head id
        sorted_keys = sorted(clusters.keys(), key=lambda x: int(x))
        self.index_to_ch = [None]  # index 0 is AUV base (O)
        self.cluster_centers = [(0.0, 0.0, 0.0)]
        for k in sorted_keys:
            c = clusters[k]['center']
            self.cluster_centers.append(tuple(c))
            self.index_to_ch.append(clusters[k].get('cluster_head', None))

        self.n = len(self.cluster_centers)

        defaults = {
            'pop_size': 50,
            'generations': 200,
            'crossover_rate': 0.8,
            'mutation_rate': 0.2,
            'elitism_k': 3,
            'tournament_size': 3,
            'crossover_type': 'OX',
            'mutation_type': 'inversion',
            'local_search': True,
            'v_f': 1.2,
            'v_AUV': 3.0,
            'verbose': False
        }
        if ga_params:
            defaults.update(ga_params)
        self.params = defaults
        self.best_fitness_history = []

    def create_individual(self):
        seq = list(range(1, self.n))
        random.shuffle(seq)
        return [0] + seq

    def create_population(self):
        return [self.create_individual() for _ in range(self.params['pop_size'])]

    def fitness(self, ind):
        total_time = travel_time(ind, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
        return 1.0 / (total_time + 1e-9)

    def tournament_selection(self, population):
        return max(random.sample(population, self.params['tournament_size']), key=self.fitness)

    def order_crossover(self, p1, p2):
        sub1, sub2 = p1[1:], p2[1:]
        a, b = sorted(random.sample(range(len(sub1)), 2))
        c1, c2 = [-1]*len(sub1), [-1]*len(sub1)
        c1[a:b], c2[a:b] = sub1[a:b], sub2[a:b]
        ptr = b
        for x in sub2[b:]+sub2[:b]:
            if x not in c1:
                c1[ptr % len(sub1)] = x
                ptr += 1
        ptr = b
        for x in sub1[b:]+sub1[:b]:
            if x not in c2:
                c2[ptr % len(sub2)] = x
                ptr += 1
        return [0]+c1, [0]+c2

    def inversion_mutation(self, ind):
        i, j = sorted(random.sample(range(1, len(ind)), 2))
        ind[i:j+1] = list(reversed(ind[i:j+1]))
        return ind

    def evolve(self):
        pop = self.create_population()
        best = max(pop, key=self.fitness)
        best_time = travel_time(best, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
        for _ in range(self.params['generations']):
            fitnesses = [self.fitness(ind) for ind in pop]
            best_gen = pop[np.argmax(fitnesses)]
            gen_best_time = travel_time(best_gen, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
            if gen_best_time < best_time:
                best, best_time = best_gen.copy(), gen_best_time
            elite_idx = np.argsort(fitnesses)[-self.params['elitism_k']:]
            new_pop = [pop[i].copy() for i in elite_idx]
            while len(new_pop) < self.params['pop_size']:
                p1, p2 = self.tournament_selection(pop), self.tournament_selection(pop)
                c1, c2 = self.order_crossover(p1, p2) if random.random() < self.params['crossover_rate'] else (p1.copy(), p2.copy())
                if random.random() < self.params['mutation_rate']:
                    c1 = self.inversion_mutation(c1)
                if random.random() < self.params['mutation_rate']:
                    c2 = self.inversion_mutation(c2)
                new_pop += [c1, c2]
            pop = new_pop[:self.params['pop_size']]
        # convert best (indices) to cluster head node ids sequence (map 0->'O')
        mapped_path = ['O' if idx == 0 else self.index_to_ch[idx] for idx in best]
        return best, mapped_path, best_time


# ==========================
# HÀM NĂNG LƯỢNG (giữ nguyên công thức bạn cung cấp)
# ==========================

def compute_energy(best_time):
    G, L, n = 100, 1024, 4
    P_t, P_r, P_idle, DR, DR_i = 1.6e-3, 0.8e-3, 0.1e-3, 4000, 1e6
    E_tx_MN = G * P_t * L / DR
    E_idle_MN = (best_time - G * L / DR) * P_idle
    E_total_MN = E_tx_MN + E_idle_MN
    E_rx_TN = G * P_r * L * n / DR
    E_tx_TN = G * P_t * L * n / DR_i
    E_idle_TN = (best_time - (G * L * n / DR) - (G * L * n / DR_i)) * P_idle
    E_total_TN = E_rx_TN + E_tx_TN + E_idle_TN
    return {
        "Member": {"E_tx": E_tx_MN, "E_idle": E_idle_MN, "E_total": E_total_MN},
        "Target": {"E_rx": E_rx_TN, "E_tx": E_tx_TN, "E_idle": E_idle_TN, "E_total": E_total_TN}
    }


# ==========================
# HÀM PHÂN CỤM VÀ CHỌN CLUSTER HEAD (sử dụng code bạn cung cấp)
# ==========================
def split_isolated_nodes(clusters, r_sen=60, node_data=None):
    """
    Tách node dựa vào khoảng cách tới Cluster Head, không dùng tâm cụm.
    Node nào xa hơn r_sen → tách thành cụm riêng.
    """

    new_clusters = []

    for c in clusters:
        nodes = c["nodes"]
        ids = c["node_ids"]

        # Lấy cluster head thật sự (không phải center)
        head_id = choose_cluster_head(c, node_data)
        head_index = ids.index(head_id)
        head_pos = nodes[head_index]

        # Tính khoảng cách node tới cluster head
        dists = np.linalg.norm(nodes - head_pos, axis=1)

        # Node trong cụm hợp lệ (gần cluster head)
        in_cluster_idx = np.where(dists <= r_sen)[0]
        # Node bị tách vì xa head
        isolated_idx = np.where(dists > r_sen)[0]

        # Cụm giữ lại (có head và các node gần head)
        if len(in_cluster_idx) > 0:
            new_clusters.append({
                "node_ids": [ids[i] for i in in_cluster_idx],
                "nodes": nodes[in_cluster_idx],
                "center": np.mean(nodes[in_cluster_idx], axis=0),
                "node_data": {nid: c["node_data"][nid] for nid in [ids[i] for i in in_cluster_idx]},
                "cluster_head": head_id   # Lưu head luôn
            })

        # Node bị tách → mỗi node thành cụm riêng
        for i in isolated_idx:
            nid = ids[i]
            node = nodes[i]
            new_clusters.append({
                "node_ids": [nid],
                "nodes": np.array([node]),
                "center": node,
                "node_data": {nid: c["node_data"][nid]},
                "cluster_head": nid  # node lẻ tự làm head
            })

    return new_clusters

def cluster_split(nodes, node_ids, node_data=None, r_sen=60, R=150, max_depth=50, depth=0):
    """
    Hàm phân cụm đệ quy (Algorithm 1) + giữ danh tính node lẻ (isolated).
    """

    center = np.mean(nodes, axis=0)
    dists = np.linalg.norm(nodes - center, axis=1)

    # Điều kiện dừng
    if (len(nodes) <= R and np.all(dists <= r_sen)) or depth >= max_depth:
        return [{
            "node_ids": node_ids,
            "nodes": nodes,
            "center": center,
            "node_data": node_data if node_data else {}
        }]

    # weights for distance vs energy
    w_dist = 0.5
    w_energy = 0.5

    # coords -> chuẩn hóa theo phạm vi hiện tại trong cụm
    coords = nodes.astype(float)
    coord_scale = np.ptp(coords, axis=0).max()  # peak-to-peak của x,y,z
    if coord_scale <= 0:
        coords_norm = np.zeros_like(coords)
    else:
        coords_norm = coords / (coord_scale + 1e-9)

    # energy feature: lấy residual_energy từ node_data nếu có, else mặc định
    if node_data:
        energies = np.array([node_data.get(nid, {}).get('residual_energy', 100.0) for nid in node_ids], dtype=float)
    else:
        energies = np.full(len(node_ids), 100.0, dtype=float)

    # chuẩn hóa năng lượng và đảo chiều nếu muốn năng lượng cao => ưu tiên (giá trị nhỏ hơn)
    emin, emax = energies.min(), energies.max()
    if emax - emin < 1e-9:
        e_norm = np.full_like(energies, 0.5)
    else:
        e_norm = 1.0 - (energies - emin) / (emax - emin)  # in [0,1]; năng lượng lớn => giá trị nhỏ

    # ghép feature với trọng số (dùng sqrt để trọng số tương ứng dưới Euclidean)
    feat = np.hstack([
        coords_norm * np.sqrt(w_dist),
        e_norm.reshape(-1, 1) * np.sqrt(w_energy)
    ])

    # nếu không đủ mẫu để chia tiếp (1 node) thì gán nhãn 0
    if len(node_ids) < 2:
        labels = np.zeros(len(node_ids), dtype=int)
    else:
        kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
        labels = kmeans.fit_predict(feat)

    clusters = []
    for i in range(2):
        sub_nodes = nodes[labels == i]
        sub_ids = [node_ids[j] for j in range(len(node_ids)) if labels[j] == i]

        sub_data = {nid: node_data[nid] for nid in sub_ids} if node_data else None

        clusters += cluster_split(sub_nodes, sub_ids, sub_data, r_sen, R, max_depth, depth + 1)

    # TÁCH NODE LẺ SAU KHI ĐỆ QUY
    clusters = split_isolated_nodes(clusters, r_sen=r_sen, node_data=node_data)

    return clusters


def choose_cluster_head(cluster, node_data_dict):
    nodes = cluster["nodes"]
    center = cluster["center"]
    node_ids = cluster["node_ids"]

    dists_to_center = np.linalg.norm(nodes - center, axis=1)
    max_Q = -1
    best_cluster_head = node_ids[0]

    for i, node_id in enumerate(node_ids):
        if node_id in node_data_dict:
            node_info = node_data_dict[node_id]
            E_current = node_info.get('residual_energy', 100.0)
            E0 = node_info.get('initial_energy', 100.0)
            if E_current <= 0:
                E_current = 0.1
            energy_ratio = E0 / E_current
            d_tocenter = dists_to_center[i]
            Q = d_tocenter ** energy_ratio
            if Q > max_Q:
                max_Q = Q
                best_cluster_head = node_id
        else:
            # fallback: choose nearest to center
            if i == 0 or dists_to_center[i] < dists_to_center[node_ids.index(best_cluster_head)]:
                best_cluster_head = node_id
    return best_cluster_head


# ==========================
# HÀM QUẢN LÝ NĂNG LƯỢNG VÀ NODE CHẾT
# ==========================

def update_energy(all_nodes, clusters, energy_report):
    # energy_report has 'Member' and 'Target' E_total
    for cid, cinfo in clusters.items():
        ch = cinfo.get('cluster_head')
        nodes = cinfo.get('nodes', [])
        for nid in nodes:
            if nid not in all_nodes:
                continue
            if nid == ch:
                all_nodes[nid]['residual_energy'] -= energy_report['Target']['E_total']
            else:
                all_nodes[nid]['residual_energy'] -= energy_report['Member']['E_total']
            # clamp
            if all_nodes[nid]['residual_energy'] < 0:
                all_nodes[nid]['residual_energy'] = 0.0


def remove_dead_nodes(all_nodes, clusters):
    dead = [nid for nid, info in list(all_nodes.items()) if info['residual_energy'] <= 0]
    for nid in dead:
        del all_nodes[nid]
    # remove from node_positions will be handled by caller
    # clean clusters: remove clusters that have no nodes left
    new_clusters = {}
    for cid, cinfo in clusters.items():
        alive_nodes = [nid for nid in cinfo.get('nodes', []) if nid in all_nodes]
        if alive_nodes:
            new_c = dict(cinfo)
            new_c['nodes'] = alive_nodes
            # if cluster head died, will select new CH during recluster
            new_clusters[cid] = new_c
    return new_clusters, dead


# ==========================
# HÀM PHÂN CỤM LẠI TỪ node_positions
# ==========================

def recluster(all_nodes, node_positions, r_sen=50, R=20):
    ids = sorted(list(all_nodes.keys()))
    if len(ids) == 0:
        return {}
    coords = np.array([node_positions[nid] for nid in ids])
    raw_clusters = cluster_split(coords, ids, all_nodes, r_sen=r_sen, R=R)
    clusters = {}
    for i, c in enumerate(raw_clusters):
        center = c['center'].tolist()
        node_ids = c['node_ids']
        ch = choose_cluster_head(c, all_nodes)
        clusters[i] = {'nodes': node_ids, 'center': center, 'cluster_head': ch}
    return clusters


# ==========================
# HÀM VẼ BIỂU ĐỒ PHÂN TÍCH
# ==========================

def plot_network_analysis(outputs, total_nodes, INITIAL_ENERGY, filename, output_dir):
    """
    Vẽ 4 biểu đồ phân tích:
    1. Số node sống theo chu kỳ
    2. Số node chết tích lũy theo chu kỳ
    3. Tổng năng lượng mạng theo chu kỳ
    4. Thời gian di chuyển AUV theo chu kỳ
    """
    
    cycles = [o['cycle'] for o in outputs]
    alive_nodes = [o['alive_nodes'] for o in outputs]
    
    # Tính số node chết tích lũy
    cumulative_dead = [total_nodes - a for a in alive_nodes]
    
    # Tính tổng năng lượng còn lại (ước tính)
    # Giả sử mỗi node bắt đầu với INITIAL_ENERGY
    # Năng lượng tiêu thụ trung bình mỗi chu kỳ
    energy_consumed_per_cycle = []
    total_energy_remaining = []
    
    for i, o in enumerate(outputs):
        # Ước tính năng lượng còn lại dựa trên số node sống
        energy_remaining = o['alive_nodes'] * INITIAL_ENERGY * (1 - (i / len(outputs)))
        total_energy_remaining.append(energy_remaining)
    
    # Thời gian di chuyển
    travel_times = [o['best_time'] for o in outputs]
    
    # Tìm chu kỳ đầu tiên có node chết
    first_death_cycle = None
    for i, o in enumerate(outputs):
        if o['dead_nodes']:
            first_death_cycle = o['cycle']
            break
    
    # Tạo figure với 4 subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f'Phân tích mạng UWSN - {filename}', fontsize=16, fontweight='bold')
    
    # Biểu đồ 1: Số node sống
    axes[0, 0].plot(cycles, alive_nodes, 'b-o', linewidth=2, markersize=6)
    axes[0, 0].axhline(y=total_nodes * 0.9, color='r', linestyle='--', 
                       label=f'Ngưỡng 90% ({int(total_nodes * 0.9)} nodes)')
    axes[0, 0].set_xlabel('Chu kỳ', fontsize=12)
    axes[0, 0].set_ylabel('Số node sống', fontsize=12)
    axes[0, 0].set_title('Số node sống theo thời gian', fontsize=14, fontweight='bold')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].legend()
    axes[0, 0].set_ylim(bottom=0)
    
    # Biểu đồ 2: Số node chết tích lũy
    axes[0, 1].plot(cycles, cumulative_dead, 'r-s', linewidth=2, markersize=6)
    axes[0, 1].set_xlabel('Chu kỳ', fontsize=12)
    axes[0, 1].set_ylabel('Số node chết (tích lũy)', fontsize=12)
    axes[0, 1].set_title('Số node chết tích lũy theo thời gian', fontsize=14, fontweight='bold')
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].set_ylim(bottom=0)
    
    # Đánh dấu chu kỳ đầu tiên có node chết
    if first_death_cycle:
        idx = first_death_cycle - 1
        axes[0, 1].axvline(x=first_death_cycle, color='orange', linestyle='--', 
                          label=f'Chu kỳ đầu tiên có node chết: {first_death_cycle}')
        axes[0, 1].legend()
    
    # Biểu đồ 3: Tổng năng lượng mạng
    axes[1, 0].plot(cycles, total_energy_remaining, 'g-^', linewidth=2, markersize=6)
    axes[1, 0].set_xlabel('Chu kỳ', fontsize=12)
    axes[1, 0].set_ylabel('Tổng năng lượng còn lại (J)', fontsize=12)
    axes[1, 0].set_title('Năng lượng tổng thể mạng theo thời gian', fontsize=14, fontweight='bold')
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].set_ylim(bottom=0)
    
    # Biểu đồ 4: Thời gian di chuyển AUV
    axes[1, 1].plot(cycles, travel_times, 'm-d', linewidth=2, markersize=6)
    axes[1, 1].set_xlabel('Chu kỳ', fontsize=12)
    axes[1, 1].set_ylabel('Thời gian di chuyển (s)', fontsize=12)
    axes[1, 1].set_title('Thời gian chu kỳ AUV', fontsize=14, fontweight='bold')
    axes[1, 1].grid(True, alpha=0.3)
    
    # Thêm thống kê
    avg_time = np.mean(travel_times)
    axes[1, 1].axhline(y=avg_time, color='r', linestyle='--', 
                      label=f'Trung bình: {avg_time:.2f}s')
    axes[1, 1].legend()
    
    plt.tight_layout()
    
    # Lưu biểu đồ
    chart_filename = os.path.join(output_dir, f"analysis_{os.path.splitext(filename)[0]}.png")
    plt.savefig(chart_filename, dpi=300, bbox_inches='tight')
    print(f"Đã lưu biểu đồ phân tích: {chart_filename}")
    
    # Hiển thị biểu đồ
    plt.show()
    plt.close()
    
    return first_death_cycle


# ==========================
# MAIN: vòng lặp nhiều chu kỳ
# ==========================

def main():
    input_dir = "D:/Year 4/tiến hóa/project/data/output_data_kmeans"
    output_dir = "D:/Year 4/tiến hóa/project/data/output_path/output_ga_multicycle"
    os.makedirs(output_dir, exist_ok=True)

    files = [f for f in os.listdir(input_dir) if f.endswith('.json')]

    ga_params = {
        'pop_size': 40,
        'generations': 200,
        'crossover_rate': 0.8,
        'mutation_rate': 0.2,
        'elitism_k': 3,
        'local_search': True,
        'v_f': 1.2,
        'v_AUV': 3.0,
        'verbose': False
    }

    INITIAL_ENERGY = 100.0

    for filename in files:
        input_path = os.path.join(input_dir, filename)
        print(f"\n=== Đang xử lý file: {filename} ===")
        with open(input_path, 'r') as f:
            clusters_in = json.load(f)

        # Build initial node list and try to find node positions
        node_positions = {}
        all_nodes = {}

        # collect all node ids from clusters_in
        all_node_ids = set()
        for k, v in clusters_in.items():
            for nid in v.get('nodes', []):
                all_node_ids.add(nid)
            ch = v.get('cluster_head')
            if ch is not None:
                all_node_ids.add(ch)

        # attempt to load node positions file (optional)
        nodes_pos_file = "D:/Year 4/tiến hóa/project/data/input_data_evenly_distributed/nodes_20.json"
        if os.path.exists(nodes_pos_file):
            try:
                with open(nodes_pos_file, 'r', encoding='utf-8') as f:
                    nodes_data = json.load(f)
            
                # Parse từ list of dict sang dict với key là node id
                node_positions = {}
                for node in nodes_data:
                    node_id = node['id']
                    node_positions[node_id] = (node['x'], node['y'], node['z'])
                
                print(f"Đã nạp node_positions từ {os.path.basename(nodes_pos_file)}")
                print(f"Số lượng node positions: {len(node_positions)}")
            except Exception as e:
                print(f"Không thể đọc {os.path.basename(nodes_pos_file)}, sẽ tạo vị trí gần center. Error:", e)

        # if no node_positions available, approximate
        if not node_positions:
            for k, v in clusters_in.items():
                center = tuple(v.get('center', (0.0, 0.0, 0.0)))
                for nid in v.get('nodes', []):
                    offset = np.random.normal(scale=5.0, size=3)
                    node_positions[nid] = tuple(np.array(center) + offset)
                ch = v.get('cluster_head')
                if ch is not None and ch not in node_positions:
                    node_positions[ch] = center
            print("Tạo giả lập node_positions bằng center + noise (vì không tìm thấy file vị trí)")

        # initialize energy
        for nid in list(all_node_ids):
            all_nodes[nid] = {'initial_energy': INITIAL_ENERGY, 'residual_energy': INITIAL_ENERGY}

        total_nodes = len(all_nodes)
        print(f"Tổng số node ban đầu: {total_nodes}")

        # Start with clusters_in
        clusters = {}
        for k, v in clusters_in.items():
            clusters[int(k)] = {'nodes': v.get('nodes', []), 'center': v.get('center', []), 
                               'cluster_head': v.get('cluster_head')}

        cycle = 0
        outputs = []
        cumulative_time = 0.0  # Tổng thời gian tích lũy

        while True:
            cycle += 1
            print(f"\n--- Cycle {cycle} ---")

            alive_ratio = len(all_nodes) / total_nodes if total_nodes > 0 else 0
            print(f"Tỉ lệ node sống: {alive_ratio*100:.2f}% ({len(all_nodes)}/{total_nodes})")
            if alive_ratio < 0.9:
                print("Dừng mô phỏng: tỉ lệ node sống < 90%")
                break

            if len(clusters) == 0:
                print("Không còn cụm nào (không còn node). Dừng.")
                break

            # run GA on current clusters
            ga = ClusterTSP_GA(clusters, ga_params)
            best_indices, best_mapped_path, best_time = ga.evolve()

            # Cộng dồn thời gian
            cumulative_time += best_time

            energy_report = compute_energy(best_time)

            # update energies
            update_energy(all_nodes, clusters, energy_report)

            # remove dead nodes
            clusters, dead_nodes = remove_dead_nodes(all_nodes, clusters)
            # also remove from node_positions
            for d in dead_nodes:
                if d in node_positions:
                    del node_positions[d]

            # Tính tổng năng lượng còn lại trong mạng
            total_energy_remaining = sum(node['residual_energy'] for node in all_nodes.values())

            # log output for this cycle
            outputs.append({
                'cycle': cycle,
                'num_clusters': len(clusters),
                'best_path_indices': best_indices,
                'best_path_node_ids': best_mapped_path,
                'best_time': best_time,
                'cumulative_time': cumulative_time,
                'dead_nodes': dead_nodes,
                'alive_nodes': len(all_nodes),
                'total_energy_remaining': total_energy_remaining
            })

            print(f"Số cụm hiện tại: {len(clusters)}")
            print(f"Đường đi (index trong GA): {best_indices}")
            print(f"Đường đi (node ids, 'O' = base): {best_mapped_path}")
            print(f"Thời gian chu kỳ này: {best_time:.2f}s")
            print(f"Tổng thời gian tích lũy: {cumulative_time:.2f}s")
            print(f"Tổng năng lượng còn lại: {total_energy_remaining:.2f}J")
            
            if dead_nodes:
                print(f"Nodes bị loại (chết) trong chu kỳ này: {dead_nodes} (Số lượng: {len(dead_nodes)})")
            else:
                print("Không có node chết ở chu kỳ này.")

            # if still nodes left, recluster and choose CH for next cycle
            if len(all_nodes) > 0:
                clusters = recluster(all_nodes, node_positions)
                # if recluster produced clusters, ensure centers are lists
                for k, v in clusters.items():
                    clusters[k]['center'] = [float(x) for x in v['center']]

        # Phân tích và tìm chu kỳ đầu tiên có node chết
        first_death_cycle = None
        first_death_time = None
        
        for o in outputs:
            if o['dead_nodes']:
                first_death_cycle = o['cycle']
                first_death_time = o['cumulative_time']
                break
        
        # In thông tin phân tích
        print("\n" + "="*60)
        print("PHÂN TÍCH KẾT QUẢ MÔ PHỎNG")
        print("="*60)
        print(f"Tổng số chu kỳ: {cycle - 1}")
        print(f"Tổng thời gian hoạt động: {cumulative_time:.2f}s ({cumulative_time/3600:.2f} giờ)")
        
        if first_death_cycle:
            print(f"\nChu kỳ đầu tiên có node chết: Chu kỳ {first_death_cycle}")
            print(f"Thời gian đến khi có node chết đầu tiên: {first_death_time:.2f}s ({first_death_time/3600:.2f} giờ)")
            print(f"Tỷ lệ thời gian: {(first_death_time/cumulative_time)*100:.2f}% tổng thời gian")
        else:
            print("\nKhông có node nào chết trong quá trình mô phỏng")
        
        print(f"\nSố node còn sống cuối cùng: {len(all_nodes)}/{total_nodes}")
        print(f"Số node đã chết: {total_nodes - len(all_nodes)}")
        print(f"Tỷ lệ sống sót: {(len(all_nodes)/total_nodes)*100:.2f}%")
        
        # Thống kê về thời gian chu kỳ
        cycle_times = [o['best_time'] for o in outputs]
        print(f"\nThời gian chu kỳ trung bình: {np.mean(cycle_times):.2f}s")
        print(f"Thời gian chu kỳ ngắn nhất: {np.min(cycle_times):.2f}s")
        print(f"Thời gian chu kỳ dài nhất: {np.max(cycle_times):.2f}s")
        print(f"Độ lệch chuẩn: {np.std(cycle_times):.2f}s")
        
        # Thống kê về số node chết mỗi chu kỳ
        deaths_per_cycle = [len(o['dead_nodes']) for o in outputs if o['dead_nodes']]
        if deaths_per_cycle:
            print(f"\nSố node chết trung bình mỗi chu kỳ (khi có chết): {np.mean(deaths_per_cycle):.2f}")
            print(f"Số node chết nhiều nhất trong 1 chu kỳ: {np.max(deaths_per_cycle)}")
        
        print("="*60)

        # save outputs to file
        out_filename = os.path.join(output_dir, f"multicycle_result_{os.path.splitext(filename)[0]}.json")
        meta = {
            'input_file': filename,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'initial_total_nodes': total_nodes,
            'cycles': cycle - 1,
            'total_operation_time': cumulative_time,
            'first_death_cycle': first_death_cycle,
            'first_death_time': first_death_time,
            'final_alive_nodes': len(all_nodes),
            'survival_rate': (len(all_nodes)/total_nodes)*100,
            'outputs': outputs
        }
        with open(out_filename, 'w') as f:
            json.dump(meta, f, indent=4)

        print(f"\nKết quả đã lưu: {out_filename}")
        
        # Vẽ biểu đồ phân tích
        print("\nĐang tạo biểu đồ phân tích...")
        plot_network_analysis(outputs, total_nodes, INITIAL_ENERGY, filename, output_dir)
        
        print("\n" + "="*60)
        print(f"Hoàn thành xử lý file: {filename}")
        print("="*60)


if __name__ == '__main__':
    main()